In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler

mort = pd.read_csv('./newdata.csv', header=0) # data about mortgage
mort.head()

## First_home 문자형 데이터 변환
mort['First_home'] = np.where(mort['First_home']=='Y',1,0)
mort['First_home']

## OUTCOME 데이터 변환
mort['OUTCOME'] = np.where(mort['OUTCOME']=='non-default',0,1) # non-default = 0, default = 1

## string 타입을 정수로 변환하기
for k in range(0,10607):
    mort.Median_state_inc[k] = np.float64(mort.Median_state_inc[k].replace(",", ""))

## Separating 'target' feature
outcome = mort.OUTCOME
mort.drop(['OUTCOME'], axis='columns', inplace=True)

mort

features = ['Bo_Age', 'Ln_Orig', 'Orig_LTV_Ratio_Pct', 'Credit_score',
            'First_home', 'Tot_mthly_debt_exp', 'Tot_mthly_incm', 'pur_prc_amt',
            'DTI Ratio', 'Median_state_inc']

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [2]:
# X, y = mort, outcome
## training data와 test data 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mort, outcome, test_size=0.3, random_state=1)

scaler = StandardScaler()
X_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [3]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(ratio='auto', kind='regular')
X_resampled, y_resampled = sm.fit_sample(X_train, y_train)

In [4]:
pd.Series(y_resampled).value_counts()

1    7216
0    7216
dtype: int64

In [5]:
from sklearn import svm

model_svm = svm.SVC(C=1000, kernel='rbf').fit(X_resampled, y_resampled)

c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [6]:
pred = model_svm.predict(X_test)

In [7]:
from sklearn.metrics import classification_report, accuracy_score
import sklearn.metrics as metrics

In [8]:
print(metrics.accuracy_score(y_test, pred))

0.976751492302859


In [9]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3109
           1       0.00      0.00      0.00        74

    accuracy                           0.98      3183
   macro avg       0.49      0.50      0.49      3183
weighted avg       0.95      0.98      0.97      3183



c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C':[1, 100, 1000],
    'kernel':['rbf']
}

results = GridSearchCV(model_svm, param_grid, cv=5).fit(X_resampled, y_resampled)

c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\svm\base.py:193: FutureWarni

In [11]:
pred = results.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      3109
           1       0.00      0.00      0.00        74

    accuracy                           0.98      3183
   macro avg       0.49      0.50      0.49      3183
weighted avg       0.95      0.98      0.97      3183



c:\anaconda3\envs\dm2019\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
